In [1]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

In [2]:
def rel_poiss(lambd):
    return -np.log(1 - st.uniform.rvs()) / lambd

def rel_poiss_proc(lambd, t):
    sum_ = rel_poiss(lambd)
    res = 0
    while (t > sum_):
        sum_ += rel_poiss(lambd)
        res += 1
    return res

In [3]:
#initialization
def control_T_tmp(que_num, service_time, init_T, max_T):
    res = 0
    if que_num > (init_T / service_time + 1):
        res = que_num * service_time
        if res > max_T:
            res = max_T
    else:
        res = init_T
    return res

def main_func_tmp(x_1=0, x_2=0, intens_1=0.9, intens_2=0.9, t_1=0.5, t_2=0.5, T1=10, T2=2, T3=10, T4=2, steps_num=1000, control=False, print_=False):

    total_working_time = (T1 + T2 + T3 + T4) * steps_num
    init_T1 = T1
    init_T3 = T3
    if control:
        total_working_time = (2*T1 + T2 + 2*T3 + T4) * steps_num
    
    #init first process
    comming_time_1 = np.zeros(int(total_working_time * intens_1 * 2))

    current_comming_el = x_1
    while comming_time_1[current_comming_el] < total_working_time:
        current_comming_el += 1
        comming_time_1[current_comming_el] = comming_time_1[current_comming_el - 1] + rel_poiss(intens_1)
        
    comming_time_1 = comming_time_1[1:current_comming_el + 1]
    exit_time_1 = comming_time_1.copy()    
    
    # init second process
    comming_time_2 = np.zeros(int(total_working_time * intens_2 * 2))

    current_comming_el = x_1
    while comming_time_2[current_comming_el] < total_working_time:
        current_comming_el += 1
        comming_time_2[current_comming_el] = comming_time_2[current_comming_el - 1] + rel_poiss(intens_2)
    comming_time_2 = comming_time_2[1:current_comming_el + 1]
    exit_time_2 = comming_time_2.copy()        
    
    #service cycle
    current_process_time = 0
    service_index_1 = 0
    service_index_2 = 0
    que_num_1 = x_1
    que_num_2 = x_2
    while current_process_time < total_working_time:
        
        #service first process
        T1_last = current_process_time + T1
        while current_process_time < T1_last and service_index_1 != len(comming_time_1):
            if current_process_time <= comming_time_1[service_index_1]:
                if comming_time_1[service_index_1] + t_1 > T1_last + T2:
                    break
                current_process_time = comming_time_1[service_index_1] + t_1
            else:
                current_process_time += t_1
            exit_time_1[service_index_1] = current_process_time
            service_index_1 += 1
            
        que_num_1 = len(comming_time_1[comming_time_1 < current_process_time]) - service_index_1
        if control:
            T1 = control_T_tmp(que_num_1, t_1, init_T1, 2*init_T1)
        if print_:
            print("G(1) - ", que_num_1, que_num_2, T1, T3)

        current_process_time = T1_last + T2
        
        #service second process
        T3_last = current_process_time + T3
        while current_process_time < T3_last and service_index_2 != len(comming_time_2):
            if current_process_time <= comming_time_2[service_index_2]:
                if comming_time_2[service_index_2] + t_2 > T3_last + T4:
                    break
                current_process_time = comming_time_2[service_index_2] + t_2
            else:
                current_process_time += t_2
            exit_time_2[service_index_2] = current_process_time
            service_index_2 += 1

        que_num_2 = len(comming_time_2[comming_time_2 < current_process_time]) - service_index_2
        if control:
            T3 = control_T_tmp(que_num_2, t_2, init_T3, 2*init_T3)
        if print_:
            print("G(3) - ", que_num_1, que_num_2, T1, T3)      
        current_process_time = T3_last + T4    
    
    return np.average((exit_time_1 - comming_time_1)[0:service_index_1]), np.average((exit_time_2 - comming_time_2)[0:service_index_2]), T1, T3

In [6]:
# different approach of searching parameters
T_init=15
T_max=40

lam_1 = 0.9
lam_2 = 0.9
best = (100000, 100000, 100000)
for t1 in np.linspace(T_init, T_max, 20): 

#     aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t1, steps_num=1000)
#     metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
    tmp_arr = []        
    for i in range(10):
        aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t1, steps_num=2000)
        tmp_arr.append(lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2)
    metric = sum(tmp_arr) / float(len(tmp_arr))
    print(t1, t1, metric)
    if metric < best[0]:
        best= (metric, t1, t1)
        print("best - " ,best)
best

15.0 15.0 700.1394197612419
best -  (700.1394197612419, 15.0, 15.0)
16.315789473684212 16.315789473684212 139.2153287558769
best -  (139.2153287558769, 16.315789473684212, 16.315789473684212)
17.63157894736842 17.63157894736842 34.19256436637829
best -  (34.19256436637829, 17.63157894736842, 17.63157894736842)
18.94736842105263 18.94736842105263 67.38688637844312
20.263157894736842 20.263157894736842 32.531196615795885
best -  (32.531196615795885, 20.263157894736842, 20.263157894736842)
21.578947368421055 21.578947368421055 24.531378008247128
best -  (24.531378008247128, 21.578947368421055, 21.578947368421055)
22.894736842105264 22.894736842105264 29.275682845663
24.210526315789473 24.210526315789473 24.504927588282197
best -  (24.504927588282197, 24.210526315789473, 24.210526315789473)
25.526315789473685 25.526315789473685 21.670857442303365
best -  (21.670857442303365, 25.526315789473685, 25.526315789473685)
26.842105263157897 26.842105263157897 24.9804617255137
28.157894736842106 28

(21.670857442303365, 25.526315789473685, 25.526315789473685)

In [11]:
# different approach of searching parameters of adaptive
T_init=15
T_max=40

lam_1 = 0.9
lam_2 = 0.9
best = (100000, 100000, 100000)
for t1 in np.linspace(T_init, T_max, 20): 

#     aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t1, steps_num=1000)
#     metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
    tmp_arr = []  
    print(t1, t1)
    for i in range(10):
        aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t1, steps_num=1000, control=True)
        tmp_arr.append(lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2)
    metric = sum(tmp_arr) / float(len(tmp_arr))
    print(t1, t1, metric)
    if metric < best[0]:
        best= (metric, t1, t1)
        print("best - " ,best)
best

15.0 15.0
15.0 15.0 48.325558054671944
best -  (48.325558054671944, 15.0, 15.0)
16.315789473684212 16.315789473684212
16.315789473684212 16.315789473684212 40.21461966630404
best -  (40.21461966630404, 16.315789473684212, 16.315789473684212)
17.63157894736842 17.63157894736842
17.63157894736842 17.63157894736842 30.050342955603224
best -  (30.050342955603224, 17.63157894736842, 17.63157894736842)
18.94736842105263 18.94736842105263
18.94736842105263 18.94736842105263 36.50899925287407
20.263157894736842 20.263157894736842
20.263157894736842 20.263157894736842 28.287870732658973
best -  (28.287870732658973, 20.263157894736842, 20.263157894736842)
21.578947368421055 21.578947368421055
21.578947368421055 21.578947368421055 23.41424804830896
best -  (23.41424804830896, 21.578947368421055, 21.578947368421055)
22.894736842105264 22.894736842105264
22.894736842105264 22.894736842105264 28.138218782613365
24.210526315789473 24.210526315789473
24.210526315789473 24.210526315789473 24.0652570041

(21.70930162490206, 25.526315789473685, 25.526315789473685)

In [12]:
# advantage adaptive alg
T1= 25.526315789473685
T3= 25.526315789473685
lam_1 = 0.9
lam_2 = 0.9
aver_1, aver_2, T1_, T3_ = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=3000, control=True)

tmp_arr = []
for i in range(10):
    aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=3000, control=True)
    tmp_arr.append(lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2)
metric = sum(tmp_arr) / float(len(tmp_arr))
print("adaprive average time - ", metric)

T1= 25.526315789473685
T3= 25.526315789473685
lam_1 = 0.9
lam_2 = 0.9
tmp_arr = []
for i in range(10):
    aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=3000)
    tmp_arr.append(lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2)
metric = sum(tmp_arr) / float(len(tmp_arr))
print("cyclic    average time - ", metric)

adaprive average time -  21.85291963714345
cyclic    average time -  21.98697958114571


In [5]:
# adaptive vs cyclic with optimal param
T1=10
T3=10
lam_1 = 0.9
lam_2 = 0.9
aver_1, aver_2, T1_, T3_ = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=1000, control=True, print_=False)
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Control -", metric, T1_, T3_)

T1=19.59
T3=19.59
lam_1 = 0.9
lam_2 = 0.9
aver_1, aver_2, T1_, T3_ = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=2000)
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Control -", metric, T1_, T3_)


Control - 69.46981898881404 13.0 16.0
Control - 25.94474355979508 19.59 19.59


In [6]:
#find optimal parameters cyclic algorithm
T1_init=10
T3_init=10
T1_max=20
T3_max=20

lam_1 = 0.9
lam_2 = 0.9
best = (10000, 100000, 100000, 100000, 100000)
for t1 in np.linspace(T1_init, T1_max, 10): 
    for t2 in np.linspace(T3_init, T3_max, 10):
        aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t2, steps_num=1000)
        metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
        print(t1, t2, metric)
        if metric < best[2]:
            best= (aver_1, aver_2, metric, t1, t2)
            print("best - " ,best)
best

10.0 10.0 1872.5215031128469
best -  (1833.6123787497816, 1911.4306274759122, 1872.5215031128469, 10.0, 10.0)
10.0 11.11111111111111 1421.3679970163355
best -  (2785.206814210224, 57.52917982244692, 1421.3679970163355, 10.0, 11.11111111111111)
10.0 12.222222222222221 1932.705624906694
10.0 13.333333333333334 2619.2849663811025
10.0 14.444444444444445 3048.249577698779
10.0 15.555555555555555 3560.5677483510026
10.0 16.666666666666668 4167.358634485327
10.0 17.77777777777778 4784.947455978169
10.0 18.88888888888889 5232.508582961361
10.0 20.0 5800.429699309655
11.11111111111111 10.0 1475.4142902002152
11.11111111111111 11.11111111111111 644.4151295714572
best -  (684.5351873993507, 604.2950717435638, 644.4151295714572, 11.11111111111111, 11.11111111111111)
11.11111111111111 12.222222222222221 874.6356026967302
11.11111111111111 13.333333333333334 1445.8551559593295
11.11111111111111 14.444444444444445 1992.7831722932954
11.11111111111111 15.555555555555555 2528.7047086376388
11.11111111

(40.135251258083045,
 35.263472921238815,
 37.69936208966093,
 17.77777777777778,
 17.77777777777778)

In [ ]:
best -  (39.54897193506841, 39.77518110326119, 40.34366191016917, 16.315789473684212, 16.315789473684212)

In [ ]:
# adaptive vs cyclic with optimal param
T1=10
T3=10
lam_1 = 0.9
lam_2 = 0.9
for i in range(10):
    aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t1, steps_num=2000, control=True)
    tmp_arr.append(lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2)
metric = sum(tmp_arr) / float(len(tmp_arr))
print(t1, t1, metric)
print("Control -", metric, T1_, T3_)

T1=19.59
T3=19.59
lam_1 = 0.9
lam_2 = 0.9
aver_1, aver_2, T1_, T3_ = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=2000)
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Control -", metric, T1_, T3_)

In [ ]:
# different approach of searching parameters of adaptive
T_init=15
T_max=20

lam_1 = 0.9
lam_2 = 0.9
best = (10000, 100000, 100000, 100000, 100000)
for t1 in np.linspace(T_init, T_max, 5): 

#     aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t1, steps_num=1000)
#     metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
    tmp_arr = []        
    for i in range(10):
        aver_1, aver_2, tmp1, tmp2 = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t1, steps_num=1000, )
        tmp_arr.append(lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2)
    metric = sum(tmp_arr) / float(len(tmp_arr))
    print(t1, t1, metric)
    if metric < best[2]:
        best= (aver_1, aver_2, metric, t1, t1)
        print("best - " ,best)
best

In [7]:
T1=10
T3=10
lam_1 = 1
lam_2 = 1
aver_1, aver_2, T1_, T3_ = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=1000, control=True)
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Control -", metric, T1_, T3_)

T1=16.666666
T3=16.666666
aver_1, aver_2, T1_, T3_ = main_func_tmp(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=1000 )
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Cyclic -", metric, T1_, T3_)

Control - 1998.21099940131 20 20
Cyclic - 1628.505291303636 16.666666 16.666666


In [243]:
class StreamModel():
    
    def __init__(self, init_que_num, service_time, intense):
        self.que_num = init_que_num
        self.service_time = service_time
#         self.distrib = lambda x: st.poisson(intense * x)
        self.total_que_num = init_que_num
        self.mesure_num = 1
        self.total_que_time = 0.0
        self.came_num = 0.0
        self.distrib = lambda x: rel_poiss_proc(intense, x)
    
    def move(self, working_time):
        time_sum = 0
        while (time_sum < working_time):
            self.service(self.service_time)
            time_sum += self.service_time
    
    def service(self, working_time):
        if (self.que_num > 0):
            self.que_num -= 1
#         new_num = self.distrib(working_time).ppf(st.uniform.rvs())
        new_num = self.distrib(working_time)
        self.total_que_time += self.que_num * working_time
        self.que_num += new_num
        self.came_num += new_num
        self.total_que_num += self.que_num
        self.mesure_num += 1
         
#         self.que_num += self.distrib(working_time)
    
    def waiting(self, working_time):
#         new_num = self.distrib(working_time).ppf(st.uniform.rvs())
        new_num = self.distrib(working_time)
        self.total_que_time += self.que_num * working_time
        self.que_num += new_num
        self.came_num += new_num
        self.total_que_num += self.que_num
        self.mesure_num += 1
        
#         self.que_num += self.distrib(working_time).ppf(st.uniform.rvs(size=1)[0])
#         self.que_num += self.distrib(working_time)
    

In [244]:
def control_T(G, init_T, max_T):
    res = 0
    if G.que_num > (init_T / G.service_time + 1):
        res = G.que_num * G.service_time
        if res > max_T:
            res = max_T
    else:
        res = init_T
    return res

In [245]:
#initialization
def main_func(x_1=0, x_2=0, intens_1=0.9, intens_2=0.9, t_1=0.5, t_2=0.5, T1=10, T2=2, T3=10, T4=2, steps_num=5000, control=False):
    G1 = StreamModel(x_1, t_1, intens_1)
    G2 = StreamModel(x_2, t_2, intens_2)
    T1_init = T1
    T3_init = T3
    
    for i in range(1, steps_num):
        p = i % 4
        if p == 1:
            G1.move(T1)
            if control and T1 != 2* T1_init: 
                T1 = control_T(G1, T1_init, 2* T1_init)
            G2.waiting(T1)
        elif p == 2:
            G1.service(T2)
            G2.waiting(T2)
        elif p == 3:
            G2.move(T3)
            if control and T3 != 2* T3_init: 
                T3 = control_T(G2, T3_init, 2* T3_init)
            G1.waiting(T3)
        else:
            G2.service(T4)
            G1.waiting(T4)            
#         print(p, G1.que_num, G2.que_num, sep="       ")
#     print("Aver len queue 1 =", float(G1.total_que_num) / G1.mesure_num)
#     print("Aver len queue 2 =", float(G2.total_que_num) / G2.mesure_num)
#     print("Aver waiting time 1 =", float(G1.total_que_time) / float(G1.came_num))
#     print("Aver waiting time 2 =", float(G2.total_que_time) / float(G2.came_num))
    return float(G1.total_que_time) / float(G1.came_num), float(G2.total_que_time) / float(G2.came_num), T1, T3

In [247]:
main_func(steps_num=4000)

(470.3329966948219, 307.2414082145851, 10, 10)

In [108]:
T1=10
T3=10
lam_1 = 0.9
lam_2 = 0.9
aver_1, aver_2, T1_, T3_ = main_func(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=10000, control=True)
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Control -", metric, T1_, T3_)

T1=16.666666
T3=16.666666
aver_1, aver_2, T1_, T3_ = main_func(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=10000 )
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Cyclic -", metric, T1_, T3_)

Control - 20.48829811917706 20 20
Cyclic - 19.072452178072755 16.666666 16.666666


In [8]:
T1=15
T3=15
lam_1 = 0.5
lam_2 = 0.5
aver_1, aver_2, T1_, T3_ = main_func(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=10000, control=True)
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Control -", metric, T1_, T3_)

T1=26.666666
T3=26.666666
aver_1, aver_2, T1_, T3_ = main_func(intens_1=lam_1, intens_2=lam_2, T1=T1, T3=T3, steps_num=10000 )
metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
print("Cyclic -", metric, T1_, T3_)

Control - 3.7440917854256197 15 15
Cyclic - 4.73189843423482 26.666666 26.666666


In [10]:
T1_init=5
T3_init=5
T1_max=15
T3_max=15

lam_1 = 0.5
lam_2 = 0.5
best = (10000, 100000, 100000, 100000, 100000)
for t1 in np.linspace(T1_init, T1_max, 10): 
    for t2 in np.linspace(T3_init, T3_max, 10):
        aver_1, aver_2, tmp1, tmp2 = main_func(intens_1=lam_1, intens_2=lam_2, T1=t1, T3=t2, steps_num=500)
        metric = lam_1 /(lam_1 + lam_2) * aver_1 + lam_2 /(lam_1 + lam_2) * aver_2
        print(t1, t2, metric)
        if metric < best[2]:
            best= (aver_1, aver_2, metric, t1, t2)
            print("best - " ,best)
best

5.0 5.0 3.0482380010106036
best -  (3.0517799352750807, 3.0446960667461265, 3.0482380010106036, 5.0, 5.0)
5.0 6.111111111111111 2.740806841719621
best -  (3.186988936988938, 2.2946247464503045, 2.740806841719621, 5.0, 6.111111111111111)
5.0 7.222222222222222 3.101170587676548
5.0 8.333333333333334 3.678546158015174
5.0 9.444444444444445 3.972881539984649
5.0 10.555555555555555 6.513539570266175
5.0 11.666666666666668 6.2472291275942196
5.0 12.777777777777779 17.849969311792194
5.0 13.88888888888889 35.86435016868419
5.0 15.0 65.89465812623033
6.111111111111111 5.0 3.186554016202619
6.111111111111111 6.111111111111111 2.95162424587709
6.111111111111111 7.222222222222222 2.8560529181411916
6.111111111111111 8.333333333333334 3.099570730878506
6.111111111111111 9.444444444444445 3.974063693918998
6.111111111111111 10.555555555555555 3.480024713276381
6.111111111111111 11.666666666666668 3.8856052137737263
6.111111111111111 12.777777777777779 3.8711043388424144
6.111111111111111 13.8888888

(3.186988936988938,
 2.2946247464503045,
 2.740806841719621,
 5.0,
 6.111111111111111)

In [369]:
poiss = st.poisson(0.5*0.1)
x = np.linspace(0, 1, 1000)

sum_=0
for i in range(100):
    sum_ +=poiss.ppf(st.uniform.rvs(size=1)[0])
sum_

12.0

In [397]:
sum_11=[]
for j in range(1000):
    sum_ = 0
    for i in range(100):
        sum_ +=poiss.ppf(st.uniform.rvs(size=1)[0])
    sum_11.append(sum_)

float(sum(sum_11)) / len(sum_11)

4.87

In [185]:
def rel_poiss(lambd):
    return -np.log(1 - st.uniform.rvs(size=1)[0]) / lambd

def rel_poiss_proc(lambd, t):
    sum_ = rel_poiss(lambd)
    res = 0
    while (t > sum_):
        sum_ += rel_poiss(lambd)
        res += 1
    return res

In [275]:
rel_poiss_proc(0.5, 0.1)

0

In [396]:

sum_11=[]
for j in range(1000):
    sum_ = 0
    for i in range(100):
        sum_ +=rel_poiss_proc(0.5, 0.1)
    sum_11.append(sum_)

float(sum(sum_11)) / len(sum_11)

5.133